<a href="https://colab.research.google.com/github/ambideXtrous9/T5-FineTuned-Model-for-NewsQA/blob/main/Context_Find_Using_BallTree%2BTF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import BallTree
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [4]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [5]:
df = df.iloc[:50000]

In [6]:
# vectorize text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['question']).toarray()

# train BallTree model with custom cosine distance metric
k = 3
tree = BallTree(X, leaf_size=k)

# **Accuaracy**

In [10]:
def predict_context(question):
    question_vect = vectorizer.transform([question]).toarray()
    distances, indices = tree.query(question_vect, k=3)
    top_context = df.iloc[indices[0][0]]['paragraph']
    return top_context

In [11]:
def pred(idx):
    ques = df.iloc[idx]['question']
    print("Q : ", ques)
    predicted_context = predict_context(ques)
    print("Actual Context : ", df.iloc[idx]['paragraph'])
    print("Predicted Context : ", predicted_context)

In [12]:
pred(0)

Q :  Who is the managing director of Synergee Capital?
Actual Context :   "Investors can use a combination of government bonds and low expense passive debt funds to build a ladder for their portfolio," says Vikram Dalal, managing director, Synergee Capital. Financial planners said investors could mix short-term, medium-term and long-term government bonds with some mutual fund products. Typically, a three-month Treasury Bill would give investors a yield of 3.5%, a one-year T bill would yield 4.11% and a 10-year government security yields 6.38%. Government bonds maturing in 2043 return 6.95% and 30- and 40-year bonds yield 6.96% and 7%, respectively. Among low-cost passive products, Bharat Bond Exchange Traded Fund - an product that invests in bonds of public sector companies - yields between 4.72% and 6.78% depending on the maturity.
Predicted Context :   Bond dealers said yields can spike in the range of 25-100 basis points. A basis point is 0.01 percentage point. Depending on the quan

In [ ]:
predict = df['question'].apply(predict_context)

In [ ]:
res = (predict == df['paragraph']).value_counts()
res

In [ ]:
accuracy = (res[True] / len(predict))*100
accuracy